In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
import tensorflow as tf

from tensorflow.keras import datasets, models, optimizers
from tensorflow.keras.layers import TimeDistributed, Conv2D, Flatten, Dense, LSTM, ConvLSTM2D, MaxPool2D, Dropout, Conv1D
import matplotlib.pyplot as plt

import numpy as np 
import pandas as pd

In [4]:
tf.__version__

'1.15.2'

In [5]:
 tf.test.is_gpu_available()

True

In [6]:
tf.test.is_built_with_cuda()

True

In [7]:
window_size = 12*60

In [8]:
train = pd.read_csv('train_timeframes3.csv').iloc[window_size-1:]
test = pd.read_csv('test_timeframes3.csv').iloc[window_size-1:]

In [9]:
df = [train, test]

In [10]:
idx = 0
batch_size = 32
margin_size = 50
time_limit = 60

In [11]:
def get_model():
    model = models.Sequential()
    # model.add(LSTM(64, input_shape = img.shape, return_sequences=True, dropout=0.2))
    model.add(LSTM(32, dropout=0.5))
    # model.add(LSTM(32, dropout=0.2))
    model.add(Dense(1, activation="linear"))
    return model

model = get_model()

In [12]:
buffer_train = time_limit
buffer_test = time_limit

In [13]:
buffer_train, buffer_test

(60, 60)

In [14]:
y_true = (df[0]['close_1min'].values[time_limit:] - df[0]['close_1min'].values[:-time_limit])/df[0]['close_1min'].values[:-time_limit]

In [15]:
model = get_model()

In [16]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.0005, epsilon=1.0),
    loss='msle',
#     loss=custom_loss_wrapper(idx, stop_limit, price_limit, margin_size),
#     metrics=[custom_metric_wrapper(idx, stop_limit, price_limit, margin_size, time_limit)],
    run_eagerly=False
)

In [33]:
def load_data(ID, path):
    return np.load(f'{path}/' + str(ID) + '.npy')

In [34]:
import numpy as np
from tensorflow.python.keras.utils.data_utils import Sequence

class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=128, n_classes=1, shuffle=True, path='inputs_train'):
        'Initialization'
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.on_epoch_end()
        self.path = path

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, window_size, 5))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = load_data(ID, self.path)

            # Store class
            y[i] = self.labels[ID]

        return X, y

In [23]:
partition = list(range(len(y_true)))
labels = y_true

# Generators
training_generator = DataGenerator(partition, labels)

model.fit_generator(
    generator=training_generator,
    epochs=10000, 
    verbose=1,
#     validation_data=validation_generator,
    use_multiprocessing=False,
    workers=2,
    max_queue_size=2,
    initial_epoch=9,
)

Epoch 10/10000
 911/7338 [==>...........................] - ETA: 1:27:11 - loss: 1.5384e-06

KeyboardInterrupt: 

In [20]:
model.save('profit_model_fast_QC_return_big')

In [1]:
model.get_config()

NameError: name 'model' is not defined

In [ ]:
model.load_weights('profit_model_fast_QC_return_big')

In [21]:
import json
weights_list = model.get_weights()
json.dump([w.tolist() for w in weights_list], open('profit_model_fast_QC_weights_return_big.json', 'w'))

In [39]:
test_y_true = (df[1]['close_1min'].values[time_limit:] - df[1]['close_1min'].values[:-time_limit])/df[1]['close_1min'].values[:-time_limit]
test_partition = list(range(len(test_y_true)))
test_labels = test_y_true
training_generator = DataGenerator(partition, labels, batch_size=10000, shuffle=False)
testing_generator = DataGenerator(test_partition, test_labels, path='inputs_test', batch_size=10000, shuffle=False)
y_preds = {'train': model.predict(training_generator), 'test': model.predict(testing_generator)}

KeyboardInterrupt: 

In [40]:
preds = model.predict(testing_generator)

In [ ]:
profits_fixed = {'train': get_profit(df[0], y_preds['train'], 0), 'test': get_profit(df[1], y_preds['test'], 0)}

In [ ]:
temp = {'train': train.iloc[:len(profits_fixed['train'])].copy(), 'test': test.iloc[:len(profits_fixed['test'])].copy()}
temp['train']['profit'] = profits_fixed['train']
temp['test']['profit'] = profits_fixed['test']

In [ ]:
temp['train']['profit'].hist()

In [ ]:
temp['test']['profit'].hist()

In [ ]:
temp['train']['profit'].plot()

In [ ]:
temp['test']['profit'].plot()

In [ ]:
temp['train']['profit'].cumsum().plot()

In [ ]:
temp['test']['profit'].cumsum().plot()

In [ ]:
sum(temp['train']['profit'])

In [ ]:
sum(temp['test']['profit'])

In [ ]:
(252 ** 0.5) * np.mean(temp['train']['profit']) / np.std(temp['train']['profit'])

In [ ]:
(252 ** 0.5) * np.mean(temp['test']['profit']) / np.std(temp['test']['profit'])